In [1]:
# Import custom libraries from local folder.
import sys
sys.path.append("..")

from calib import EventDataset
from calib.nn import ConjunctionEventForecaster as CEF
from calib.data import kelvins_to_event_dataset
import pandas as pd
import numpy as np 

np.random.seed(1)

In [69]:
# Import json library and create function to format dictionaries.
import json
format_json = lambda x: json.dumps(x, indent=4)

class RNNlayer():
    def __init__(self, layer_type:str, input_size:int, 
                 output_size:int, **kwargs):

        # Check layer_type is either linear or lstm.
        if not layer_type in ['lstm', 'linear']:
            raise ValueError('Layer {} not recognised'.format(layer_type))

        self.layer_type = layer_type
        self.input_size = input_size
        self.output_size = output_size

        # Set internal values for the model.
        for key, value in kwargs.items(): setattr(self, '_' + key, value)


        if layer_type == 'lstm':

            default_parameters = dict(num_layers=2, batch_first=True, dropout=0)
            for parameter, value in default_parameters.items():
                if not parameter in kwargs.keys():
                    print('Parameter {} missing in the class. Setting ' \
                          'default value ({}).'.format(parameter, value))

            
            self.layer = nn.LSTM(input_size = self.input_size, 
                            hidden_size     = self.output_size, 
                            num_layers      = kwargs.get('num_layers',2), 
                            batch_first     = kwargs.get('batch_first', True), 
                            dropout         = kwargs.get('dropout', 0),
                            bias            = kwargs.get('bias', True),
                            bidirectional   = kwargs.get('bidirectional', False),
                            proj_size       = kwargs.get('proj_size', 0))

        elif layer_type == 'linear':

            self.layer = nn.Linear(in_features  = self.input_size, 
                                  out_features  = self.output_size,
                                  bias          = kwargs.get('bias', True), 
                                  device        = kwargs.get('device', None), 
                                  dtype         = kwargs.get('dtype', None))


    def __repr__(self) -> str:
        """Print readable information about the layer.

        Returns:
            str: Class name with number of CDMs objects contained on it.
        """
        return 'RNNlayer: {}'.format(self.layer)

    def __getitem__(self):
        """Get layer object.
        """
        return self.layer


def pack_padded_sequence(x, x_lengths):

    return nn.utils.rnn.pack_padded_sequence(input = x, 
                                              lengths = x_lengths, 
                                              batch_first=True, 
                                              enforce_sorted=False)

def pad_packed_sequence(x, x_length_max):

    # Pads a packed batch of variable length sequences from LSTM layer.
    x, _ = nn.utils.rnn.pad_packed_sequence(sequence = x, 
                                            batch_first=True, 
                                            total_length = x_length_max)
    return x   


# Define input and output size of the RNN model.
input_size = 50
output_size = 25

# Define parameter batch_first for batch processing.
batch_first = True

# Define number of output neurons per layer.
layers = [dict(layer_type = 'linear', 
               output_size = 100), 
          dict(layer_type = 'lstm', 
                output_size = 200, 
                batch_first = batch_first,
                num_layers = 2,
                dropout = 0.2), 
          dict(layer_type = 'linear', 
               output_size = 100)]

layerlist = []
for l, layer in enumerate(layers):

    kwargs = {}
    for key, value in layer.items():
        if key in ['layer_type', 'input_size', 'output_size']: continue
        kwargs[key] = value

    if layer['layer_type']=='lstm': 
        layerlist.append(pack_padded_sequence)

    i_layer = RNNlayer(layer_type = layer['layer_type'],
                       input_size = input_size, 
                       output_size = layer['output_size'],
                       **kwargs)

    # Append i_layer object to list
    layerlist.append(i_layer)

    if layer['layer_type']=='lstm': 
        layerlist.append(pad_packed_sequence)

    # Cancel out a random proportion p of the neurons to avoid overfitting
    if 'dropout' in kwargs.keys():
        layerlist.append(nn.Dropout(kwargs.get('dropout', 0), inplace = True))

    # Apply ReLU activation function (al(z))
    layerlist.append(nn.ReLU(inplace=True))
    

    # Define input_size for the next layer.
    input_size = layer['output_size']


last_layer = RNNlayer(layer_type = 'linear', 
                   input_size = output_size,
                   output_size = output_size)

layerlist.append(last_layer)

print('Sequential of layers: ')
for l, layer in enumerate(layerlist):
    print('({}) {}'.format(l, layer))



Sequential of layers: 
(0) RNNlayer: Linear(in_features=50, out_features=100, bias=True)
(1) ReLU(inplace=True)
(2) <function pack_padded_sequence at 0x11ba7d8b0>
(3) RNNlayer: LSTM(100, 200, num_layers=2, batch_first=True, dropout=0.2)
(4) <function pad_packed_sequence at 0x11ba66160>
(5) Dropout(p=0.2, inplace=True)
(6) ReLU(inplace=True)
(7) RNNlayer: Linear(in_features=200, out_features=100, bias=True)
(8) ReLU(inplace=True)
(9) RNNlayer: Linear(in_features=25, out_features=25, bias=True)


# Data Loading

Kessler accepts CDMs either in KVN format or as pandas dataframes. We hereby show a pandas dataframe loading example:

In [2]:
#As an example, we first show the case in which the data comes from the Kelvins competition.
#For this, we built a specific converter that takes care of the conversion from Kelvins format
#to standard CDM format (the data can be downloaded at https://kelvins.esa.int/collision-avoidance-challenge/data/):
file_name = '/Users/jjrr/Documents/SCA-Project/calib/data/esa-challenge/train_data.csv'
events = kelvins_to_event_dataset(file_name, drop_features=['c_rcs_estimate', 't_rcs_estimate'], num_events=1000) #we use only 200 events

Loading Kelvins dataset from file name: /Users/jjrr/Documents/SCA-Project/Tool/data/esa-challenge/train_data.csv
162634 entries
Dropping features: ['c_rcs_estimate', 't_rcs_estimate']
Dropping rows with NaNs
146571 entries
Removing outliers
127037 entries
Shuffling
Grouped rows into 9586 events
Taking TCA as current time: 2023-07-21 13:22:02.766396
Converting Kelvins challenge data to EventDataset
Time spent  | Time remain.| Progress             | Events    | Events/sec
0d:00:00:04 | 0d:00:00:00 | #################### | 1000/1000 | 228.88       


In [ ]:
#Instead, this is a generic real CDM data loader that should parse your Pandas (uncomment the following lines if needed):
#file_name = 'path_to_csv/file.csv'

#df=pd.read_csv(file_name)
#events = EventDataset.from_pandas(df)

# Descriptive Statistics

In [ ]:
#Descriptive statistics of the event:
kessler_stats = events.to_dataframe().describe()
print(kessler_stats)


# LSTM Training

In [ ]:
#We only use features with numeric content for the training
#nn_features is a list of the feature names taken into account for the training:
#it can be edited in case more features want to be added or removed
nn_features = events.common_features(only_numeric=True)
print(nn_features)

In [ ]:
# Split data into a test set (5% of the total number of events)
len_test_set=int(0.05*len(events))
print('Test data:', len_test_set)
events_test=events[-len_test_set:]
print(events_test)

# The rest of the data will be used for training and validation
print('Training and validation data:', len(events)-len_test_set)
events_train_and_val=events[:-len_test_set]
print(events_train_and_val)

In [ ]:
# Create an LSTM predictor, specialized to the nn_features we extracted above
model = CEF(
            lstm_size=256,  # Number of hidden units per LSTM layer
            lstm_depth=2,  # Number of stacked LSTM layers
            dropout=0.2,  # Dropout probability
            features=nn_features)  # The list of feature names to use in the LSTM

# Start training
model.learn(events_train_and_val, 
            epochs=10, # Number of epochs (one epoch is one full pass through the training dataset)
            lr=1e-3, # Learning rate, can decrease it if training diverges
            batch_size=16, # Minibatch size, can be decreased if there are issues with memory use
            device='cpu', # Can be 'cuda' if there is a GPU available
            valid_proportion=0.15, # Proportion of the data to use as a validation set internally
            num_workers=4, # Number of multithreaded dataloader workers, 4 is good for performance, but if there are any issues or errors, please try num_workers=1 as this solves issues with PyTorch most of the time
            event_samples_for_stats=1000) # Number of events to use to compute NN normalization factors, have this number as big as possible (and at least a few thousands)

In [ ]:
#Save the model to a file after training:
model.save(file_name="models/rnn/LSTM_20epochs_lr10-4_batchsize16")

In [ ]:
#NN loss plotted to a file:
model.plot_loss(file_name='images/plot_loss.pdf')

In [ ]:
#we show an example CDM from the set:
events_train_and_val[0][0]

In [ ]:
#we take a single event, we remove the last CDM and try to predict it
event=events_test[3]
event_len = len(event)
print(event)
event_beginning = event[0:event_len-1]
print(event_beginning)
event_evolution = model.predict_event(event_beginning, num_samples=100, max_length=14)

In [ ]:
#We plot the prediction in red:
axs = event_evolution.plot_features(['RELATIVE_SPEED', 'MISS_DISTANCE', 'OBJECT1_CT_T'], return_axs=True, linewidth=0.1, color='red', alpha=0.33, label='Prediction')
#and the ground truth value in blue:
event.plot_features(['RELATIVE_SPEED', 'MISS_DISTANCE', 'OBJECT1_CT_T'], axs=axs, label='Real', legend=True)

In [ ]:
#we now plot the uncertainty prediction for all the covariance matrix elements of both OBJECT1 and OBJECT2:
axs = event_evolution.plot_uncertainty(return_axs=True, linewidth=0.5, label='Prediction', alpha=0.5, color='red', legend=True, diagonal=False)
event.plot_uncertainty(axs=axs, label='Real', diagonal=False)